In this file i used logistic regression for training speed col

In [1]:
import pandas as pd
import numpy as np

# Load the training data (You might have to change the file path based on how you're working)
train_df = pd.read_csv('BH25/Training_Data/train.csv')

print(train_df.head())

   video_id element  motion power  speed          video_summary
0         1    Erde  linear   rot    9.6     (3.8147, 31.94809)
1         2   Feuer     shm  grin    9.6   (26.70288, -4.29153)
2         3   Feuer  random  geld    9.6      (3.8147, 8.58307)
3         4    Erde  zigzag  lila    6.9  (-24.79553, -0.95367)
4         5    Erde  linear  lila    9.6    (7.62939, 22.88818)


In [2]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical attributes using the Label Encoder
def encode_categorical_columns(df, columns):
    label_encoders = {}
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    return df, label_encoders

# List of categorical columns to encode
categorical_columns = ['element', 'motion', 'power']

# Encoding the dataframe
train_df, label_encoders = encode_categorical_columns(train_df, categorical_columns)

# Print the mapping for each column
for col in categorical_columns:
    print(f"Column: {col}")
    print("Attribute -> Encoded Value")
    for attribute, value in zip(label_encoders[col].classes_, range(len(label_encoders[col].classes_))):
        print(f"{attribute} -> {value}")
    print()


Column: element
Attribute -> Encoded Value
Boden -> 0
Erde -> 1
Feuer -> 2
Luft -> 3
Wasser -> 4

Column: motion
Attribute -> Encoded Value
circular -> 0
linear -> 1
random -> 2
shm -> 3
zigzag -> 4

Column: power
Attribute -> Encoded Value
baau -> 0
blau -> 1
blru -> 2
brun -> 3
btau -> 4
gaun -> 5
gela -> 6
gelb -> 7
geld -> 8
gele -> 9
gelt -> 10
got -> 11
grin -> 12
grun -> 13
grus -> 14
helb -> 15
lela -> 16
liga -> 17
lila -> 18
lili -> 19
lisa -> 20
lula -> 21
not -> 22
plau -> 23
rat -> 24
rod -> 25
rot -> 26
rut -> 27
slau -> 28
trun -> 29



In [3]:
import cv2 

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)  # Shape: (20, 64, 64, 3)
    return frames.flatten()  # Flatten to 1D array
    
video_features = []
for idx, row in train_df.iterrows():
    video_path = f"BH25/Training_Data/Train_Videos/{row['video_id']}.mp4"
    video_features.append(process_video(video_path))

video_features = np.array(video_features)  # Shape: (num_samples, 20*64*64*3)


In [4]:
from sklearn.model_selection import train_test_split


y_speed = train_df[['speed']]
y_summary = train_df['video_summary'].apply(lambda x: eval(x))  # Convert string tuples to actual tuples

y_categorical = train_df[categorical_columns]

# Split data for training and validation
X_train_speed, X_val_speed, y_train_speed, y_val_speed = train_test_split(video_features, y_speed, test_size=0.2, random_state=42)
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(video_features, y_categorical, test_size=0.2, random_state=42)
X_train_summary, X_val_summary, y_train_summary, y_val_summary = train_test_split(video_features, y_summary, test_size=0.2, random_state=42)

In [5]:
X_batch_speed=X_train_speed[:3000]
y_batch_speed=y_train_speed[:3000]

In [6]:
#train logistic speed
from sklearn.linear_model import LogisticRegression
# Map 6.9 to 0 and 9.6 to 1
y_batch_speed_binary = np.where(y_batch_speed == 6.9, 0, 1)
# Initialize and train the model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed for convergence
model.fit(X_batch_speed, y_batch_speed_binary)

c:\Users\hp\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [7]:
#for mae of speed for logistic
from sklearn.metrics import mean_absolute_error
import numpy as np

# Example: Convert binary predictions back to continuous values
y_pred_speed_binary = model.predict(X_val_speed)  # y_pred_binary is the binary output from LogisticRegression
y_pred_speed = np.where(y_pred_speed_binary == 0, 6.9, 9.6)

# Calculate MAE
mae_continuous = mean_absolute_error(y_val_speed, y_pred_speed)

# Output the result
print("Mean Absolute Error:", mae_continuous)

Mean Absolute Error: 0.2659499999999999


In [8]:
import os

test_folder = 'BH25/Testing_Data'

test_features = []
video_ids = []

# The below function finds all the video ids in the test folder and sorts them, 
# and then stores their features in test_features

for video_id in sorted(
    [f for f in os.listdir(test_folder) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_folder, video_id)
    video_ids.append(video_id.split('.')[0]) 
    test_features.append(process_video(video_path))

test_features = np.array(test_features)  # Shape: (num_test_samples, 20*64*64*3)

In [9]:
#convert the binary output back to 6.9 and 9.6
y_test_speed_binary = model.predict(test_features)
y_test_speed = np.where(y_test_speed_binary == 0, 6.9, 9.6)

In [10]:
speed_predictions = y_test_speed.flatten()  # Ensure the shape is (num_samples,)

In [11]:
output_df = pd.DataFrame({
    'video_id': video_ids,                      
    'speed': speed_predictions,               
    
})

In [12]:
output_df.to_csv("speed.csv", index=False)